https://www.dataquest.io/blog/pandas-big-data/

In [2]:
import pandas as pd

Data is on S3: mytaxi-datascience-passenger-destination/data/processed/sorted_passenger_tours_processed.csv.zip

In [4]:
df = pd.read_csv('/Users/caiomiyashiro/repo/passenger_destination/data/raw/last_16_passenger_tours_v2.csv', index_col=0)

/Library/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (79,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


How much space does this dataframe use? (17.2 GB)

In [5]:
df.info(memory_usage='deep') 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8581762 entries, 2747 to 31942516
Data columns (total 83 columns):
date_created                   object
id                             int64
request_long                   float64
request_lat                    float64
request_street                 object
request_street_number          object
dest_long                      float64
dest_lat                       float64
dest_street                    object
dest_street_number             object
passenger_input_destination    bool
lag1_dest_long                 float64
lag1_dest_lat                  float64
lag1_street                    object
lag1_street_number             object
lag2_dest_long                 float64
lag2_dest_lat                  float64
lag2_street                    object
lag2_street_number             object
lag3_dest_long                 float64
lag3_dest_lat                  float64
lag3_street                    object
lag3_street_number             object
la

# What is the Internal Representation of a Dataframe?
  
Under the hood, a Pandas Data Frame stores all the contiguous values in different sub data structures. For numeric data structures, each sub-block is stored in a numpy array, which per se are wrappers for C arrays style, which finally makes it efficient to access and process.

The same doesn't happen for object blocks, which can be anything else besides numeric. With this, they're subject to all the Python slowliness it is subject to (Source: [Why Python is Slow?](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/))

<img src="img1.png" width="800">

Lets see what is the average column memory size for our dataframe:

In [6]:
def average_memory_use(df, dtypes=['float','int','object']):
    for dtype in dtypes:
        selected_dtype = df.select_dtypes(include=[dtype])
        mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
        mean_usage_mb = mean_usage_b / 1024 ** 2
        print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))
        
average_memory_use(df)        

Average memory usage for float columns: 65.47 MB
Average memory usage for int columns: 65.47 MB
Average memory usage for object columns: 512.24 MB


# Some stuff we can do to decrease the data size:

## Defining function to extract the column's size

In [7]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

original_df_size = mem_usage(df)
print('Original data frame memory size: ' + str(original_df_size))

Original data frame memory size: 24206.56 MB


## pd.to_numeric() to downcast numeric variables - Floats or Integers

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html

In [8]:
df_float = df.select_dtypes(include=['float'])
df_float_converted = df_float.apply(pd.to_numeric,downcast='float')

print('Memory used by all the float64:')
print(mem_usage(df_float))
print('\n')
print('New memory size used by all the float32:')
print(mem_usage(df_float_converted))

Memory used by all the float64:
2684.42 MB


New memory size used by all the float32:
1374.95 MB


# Why string objects take so much space?
  
  
Every string present in the dataframe will be repetively stored in the computer's memory, *i.e.* if I have a columns of 1 million rows with a string value in it, python will store 1 million string on memory, even though it could store just a reference to same place in memory.
  

<img src="numpy_vs_python.png" width="600">
source: https://www.dataquest.io/blog/pandas-big-data/  

## Converting string objects to category

[The category dtype](https://pandas.pydata.org/pandas-docs/stable/categorical.html) does exactly the above, it creates a numeric representation of each category value and just store in the array a reference to each value. This is the **main** memory save **if** we have a limited amount of category values of course. If we convert an id columns, it will at the end just increase the memory size because it had to create an numeric mapping and still keep all the distinct category values in memory.  
  
A rule of thumb (totally) is to turn variables in category if the amount of distinct values <= 50% of the numbers of rows. Keeping like this it is more probable we end up saving memory :)

In [9]:
df.select_dtypes(include=['object']).columns

Index(['date_created', 'request_street', 'request_street_number',
       'dest_street', 'dest_street_number', 'lag1_street',
       'lag1_street_number', 'lag2_street', 'lag2_street_number',
       'lag3_street', 'lag3_street_number', 'lag4_street',
       'lag4_street_number', 'lag5_street', 'lag5_street_number',
       'lag6_street', 'lag6_street_number', 'lag7_street',
       'lag7_street_number', 'lag8_street', 'lag8_street_number',
       'lag9_street', 'lag9_street_number', 'lag10_street',
       'lag10_street_number', 'lag11_street', 'lag11_street_number',
       'lag12_street', 'lag12_street_number', 'lag13_street',
       'lag13_street_number', 'lag14_street', 'lag14_street_number',
       'lag15_street', 'lag15_street_number', 'lag16_street',
       'lag16_street_number', 'home_street_name', 'home_street_number',
       'work_street_name', 'work_street_number'],
      dtype='object')

In [10]:
df_object = df.select_dtypes(include=['object']).copy()
# we don't want the convert date_created to category, we want to keep it as datetime
df_object.drop('date_created', axis=1,inplace=True) 

df_object_converted = df_object.apply(lambda col: col.astype('category'))

print('Memory used by all the object dtypes:')
print(mem_usage(df_object))
print('\n')
print('New memory size used by all the category dtypes:')
print(mem_usage(df_object_converted))

Memory used by all the object dtypes:
20891.96 MB


New memory size used by all the category dtypes:
1580.18 MB


In [9]:
1134.64/14973.74

0.0757753240005503

That is equal to > 90% memory reduction!
  
Below we can see the transformation is just under the hood. Up front, the data is 100% the same.

In [11]:
df_object.head()

,request_street,request_street_number,dest_street,dest_street_number,lag1_street,lag1_street_number,lag2_street,lag2_street_number,lag3_street,lag3_street_number,...,lag14_street,lag14_street_number,lag15_street,lag15_street_number,lag16_street,lag16_street_number,home_street_name,home_street_number,work_street_name,work_street_number
id_passenger,,,,,,,,,,,,,,,,,,,,,
2747,Max-Brauer-Allee,215-217,Präsident-Krahn-Straße,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65
2747,Torstraße,164,Kurfürstendamm,226,Präsident-Krahn-Straße,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65
2747,Charlottenstraße,59,Novalisstraße,10-11,Kurfürstendamm,226,Präsident-Krahn-Straße,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65
2747,Novalisstraße,11,Hannoversche Straße,23,Novalisstraße,10-11,Kurfürstendamm,226,Präsident-Krahn-Straße,6,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65
2747,Alstertwiete,1b,Hamburger Berg,37,Hannoversche Straße,23,Novalisstraße,10-11,Kurfürstendamm,226,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65


In [12]:
df_object_converted.head()

,request_street,request_street_number,dest_street,dest_street_number,lag1_street,lag1_street_number,lag2_street,lag2_street_number,lag3_street,lag3_street_number,...,lag14_street,lag14_street_number,lag15_street,lag15_street_number,lag16_street,lag16_street_number,home_street_name,home_street_number,work_street_name,work_street_number
id_passenger,,,,,,,,,,,,,,,,,,,,,
2747,Max-Brauer-Allee,215-217,Präsident-Krahn-Straße,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65
2747,Torstraße,164,Kurfürstendamm,226,Präsident-Krahn-Straße,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65
2747,Charlottenstraße,59,Novalisstraße,10-11,Kurfürstendamm,226,Präsident-Krahn-Straße,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65
2747,Novalisstraße,11,Hannoversche Straße,23,Novalisstraße,10-11,Kurfürstendamm,226,Präsident-Krahn-Straße,6,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65
2747,Alstertwiete,1b,Hamburger Berg,37,Hannoversche Straße,23,Novalisstraße,10-11,Kurfürstendamm,226,...,NaN,NaN,NaN,NaN,NaN,NaN,Stresemannstraße,108A,Zollstockgürtel,65


# Joining Every Transformation

Lets take the original dataset, apply the transformations and compare the old with the new total size:

In [13]:
df[df_float_converted.columns] = df_float_converted
df[df_object_converted.columns] = df_object_converted

new_df_size = mem_usage(df)

print('Original data frame memory size: ' + str(original_df_size))
print('New data frame memory size: ' + str(new_df_size))

Original data frame memory size: 24206.56 MB
New data frame memory size: 3585.31 MB


# During Projects

If the dataset is already too big, we wouldn't be able to read the whole file so just then decrease its size. An approach that I did was to read in a limited amount of lines and process the lines in order to identify the objects dtypes, as they are the most useless in terms of memory usage. For that we use the **nrows** parameter and them loop create a dictionary with the columns' names and their dtypes :

In [14]:
# Read only a sample to detect data types
df = pd.read_csv('/Users/caiomiyashiro/repo/passenger_destination/data/raw/last_16_passenger_tours_v2.csv', index_col=0, nrows=10000)
print(df.shape)
display(df.head())

/Library/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(10000, 83)


,date_created,id,request_long,request_lat,request_street,request_street_number,dest_long,dest_lat,dest_street,dest_street_number,...,lag16_street,lag16_street_number,home_dest_long,home_dest_lat,home_street_name,home_street_number,work_dest_long,work_dest_lat,work_street_name,work_street_number
id_passenger,,,,,,,,,,,,,,,,,,,,,
2747,2017-02-23 06:21:31,37109452,9.95446,53.56107,Max-Brauer-Allee,215-217,9.93620,53.55252,Präsident-Krahn-Straße,6,...,NaN,NaN,9.955721,53.560528,Stresemannstraße,108A,6.936007,50.906535,Zollstockgürtel,65
2747,2017-03-03 23:02:49,37616638,13.39608,52.52890,Torstraße,164,13.32981,52.50328,Kurfürstendamm,226,...,NaN,NaN,9.955721,53.560528,Stresemannstraße,108A,6.936007,50.906535,Zollstockgürtel,65
2747,2017-03-04 21:14:50,37673824,13.39109,52.51282,Charlottenstraße,59,13.38837,52.52887,Novalisstraße,10-11,...,NaN,NaN,9.955721,53.560528,Stresemannstraße,108A,6.936007,50.906535,Zollstockgürtel,65
2747,2017-03-04 23:13:06,37682372,13.38827,52.52862,Novalisstraße,11,13.33020,52.50370,Hannoversche Straße,23,...,NaN,NaN,9.955721,53.560528,Stresemannstraße,108A,6.936007,50.906535,Zollstockgürtel,65
2747,2017-04-29 00:38:11,41699227,10.00676,53.55730,Alstertwiete,1b,9.96197,53.54960,Hamburger Berg,37,...,NaN,NaN,9.955721,53.560528,Stresemannstraße,108A,6.936007,50.906535,Zollstockgürtel,65


In [14]:
dtypes_df = {}
obj_cols = df.select_dtypes(include=['object']).columns[1:].tolist() # [1:] is just to skip the 'date_created'
for obj_col in obj_cols:
    if(len(df[obj_col].drop_duplicates()) < df.shape[0]/2):
        dtypes_df[obj_col] = 'category'
dtypes_df        

{'dest_street': 'category',
 'dest_street_number': 'category',
 'home_street_name': 'category',
 'home_street_number': 'category',
 'lag10_street': 'category',
 'lag10_street_number': 'category',
 'lag1_street': 'category',
 'lag1_street_number': 'category',
 'lag2_street': 'category',
 'lag2_street_number': 'category',
 'lag3_street': 'category',
 'lag3_street_number': 'category',
 'lag4_street': 'category',
 'lag4_street_number': 'category',
 'lag5_street': 'category',
 'lag5_street_number': 'category',
 'lag6_street': 'category',
 'lag6_street_number': 'category',
 'lag7_street': 'category',
 'lag7_street_number': 'category',
 'lag8_street': 'category',
 'lag8_street_number': 'category',
 'lag9_street': 'category',
 'lag9_street_number': 'category',
 'request_street': 'category',
 'request_street_number': 'category',
 'work_street_name': 'category',
 'work_street_number': 'category'}

We use the created dictionary as input for the **pd.read_csv** function in the **dtype** parameter. Pandas won't try to infer the column's type and directly try to apply the specified format

In [15]:
df = pd.read_csv('../data/raw/sorted_passenger_tours_v2.csv', index_col=0, 
                 dtype=dtypes_df, parse_dates=['date_created'], infer_datetime_format=True)

print('Predefined formats data frame memory size: ' + str(mem_usage(df)))

/Library/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Predefined formats data frame memory size: 3225.63 MB


# Things I know it doesn't work and other questions:



- Unfortunately, when working with pandas, I know the category data type is not automatically interpreted as factors, like in R. I haven't found a solution for this besides using again lots of memory to convert the category dtypes to strings (pandas will do this when calling pd.get_dummies() anyway)
  
- If we have to deal with string manipulation, I don't know if the category dtype impact in the performance.  
  
- In case of nominal data where the categories have an order, *e.g.* 'disagree', 'agree', 'strongly agree', we can instantiate the categorical dtype in a similar way, using one extra parameter of *order = True* [Link here](https://pandas.pydata.org/pandas-docs/stable/categorical.html#controlling-behavior)